In [4]:
import os
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
from groq import Groq
import openai

import minsearch

In [3]:
load_dotenv()

True

In [26]:
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

## Ingestion

In [5]:
df = pd.read_csv('../data/data.csv')
documents = df.to_dict(orient='records')

In [6]:
documents[0]

{'id': 0,
 'exercise_name': 'Push-Up',
 'type_of_activity': 'Strength',
 'type_of_equipment': 'Bodyweight',
 'body_part': 'Upper Body',
 'type': 'Push',
 'muscle_groups_activated': 'Chest, Triceps, Shoulders, Core',
 'instructions': 'Start in a plank position, lower your body until your chest nearly touches the floor, then push back up to the starting position.'}

In [7]:
prompt_template = """
You emulate a user of our fitness assistant applications.
Formulate 5 questions this user might ask based on a provided exercise. 
The record should contain the answer to the questions, and the questions should 
be complete and not too short. Use fewer words as possible from the record. 

The record:
exercise_name : {exercise_name}
type_of_activity : {type_of_activity}
type_of_equipment : {type_of_equipment}
body_part : {body_part}
type : {type}
muscle_groups_activated : {muscle_groups_activated}
instructions : {instructions}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()